In [ ]:
minsizes=list(range(3,50))
import hdbscan
import umap
reducer = umap.UMAP( n_components=3,random_state=0)
embedding = reducer.fit_transform(embeddings)
bedded2=pd.DataFrame({'X1':embedding[:,0],
                      "X2":embedding[:,1],
                      "X3":embedding[:,2]})


coherencescores=[]
noveltyscores=[]
for i in minsizes:
    clusterer = hdbscan.HDBSCAN(min_cluster_size=i)
    cluster_labels = clusterer.fit_predict(bedded2[['X1','X2','X3']])
    LabFrame=pd.DataFrame({'Cluster':cluster_labels.astype('str'),'Item':forjoin})
    LabFrame=LabFrame[LabFrame['Cluster']!='-1']
    topsr=get_ngrams(LabFrame.drop_duplicates(['Cluster','Item']).set_index('Cluster')['Item'],1).reset_index().\
    assign(Dummy=1).groupby(['Cluster','Item'])['Dummy'].sum().reset_index().\
    sort_values(['Cluster','Dummy'],ascending=False).groupby(['Cluster']).head(5).\
    reset_index(drop=True)
    tested=topsr.groupby('Cluster')['Item'].apply(list).reset_index()
    
    coherence=[]
    for zed in tested['Item']:
        coherence.append(word2vecscore(zed,model))
    
    coherencescores.append(np.median(coherence))
    
    
    alltopics=list(itertools.combinations(list(tested['Item']),2))
    redundancy=[]
    for j in alltopics:
        redundancy.append(model.wv.n_similarity(j[0],j[1]))
    
    noveltyscores.append(1-np.median(redundancy))
    

pd.DataFrame({'MinSize':minsizes,
              'Coherence':coherencescores,
                'Novel':noveltyscores}).plot(x='MinSize')
print(pd.DataFrame({'MinSize':minsizes,
              'Coherence':coherencescores,
                'Novel':noveltyscores}))    